In [92]:
import json
import requests
import time
import urllib.request
import os
if not os.path.exists('new_url'):
    os.makedirs('new_url')

In [93]:
# set image parameters
img1 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_339187295.jpg"
img2 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_2_339187295.jpg"
img3 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_3_339187295.jpg"
img4 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_4_339187295.jpg"
img5 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_5_339187295.jpg"
img_lst = [img1,img2,img3,img4,img5]

In [94]:
def json_builder(img_lst):
    dealerId = 1
    vid = 100
    dct = {}
    for url in img_lst:
        my_json_string = {'dealerId':dealerId,'vehicles':[{"id":vid,"images":[{"imageId":vid,"imageUrl":url}]}]}
        dct[dealerId] = my_json_string
        dealerId +=1
        vid = dealerId + 100
    return dct

In [95]:
json_dct = json_builder(img_lst)

In [96]:
# send request
token_status = {}
for k,v in json_dct.items():
    json_body = v
    r = requests.post('https://photoai.lotlinx.com/images/optimize', json=json_body,auth=('testaccount5','984dc25e4e50'))
    rdct= r.json()
    token = rdct['data'][0]['token']
    status = rdct['data'][0]['status']
    token_status[token] = status

In [97]:
token_status

{'NAaWW8jSkFBoEHqi39W5xBnuMQJnUUwm0QvG34Imt1cFmaS2wbrdGYEMhwAgktpu': 'queued',
 'OBBh7FDrHJbfYgYudHAoPxTgR4wD5nCoPZFS06eAtiDF4B4PiD11can0th4SDNVu': 'queued',
 'QawnDoVtCP5kpZX92NfX9iKof19tqJxzcu8UZMKAca4dI795q1M8hwVKkSuzSBqY': 'queued',
 'nvBC8L9ad3PGti6Os5jrPLfdYjExmWytgF4kTgMROGF4UFPGMG3IUPwOlB8LFZ3Z': 'queued',
 'zzOhhc3JQOqumCTujipynSq1VX54muovfMNHcEFhhvJpvHa9TM9qPLtQm048nhwY': 'queued'}

In [98]:
def poll_api():
    count = 0
    stme = time.time()
    token_completed = []
    j=0
    #token_status_poll = {}
    print ("Status poll has started at ", stme)
    while count <= 30: # poll for 1.5 min max
        for k,v in token_status.items():
            r = requests.get('https://photoai.lotlinx.com/images/{}/status'.format(k), auth=('testaccount5','984dc25e4e50'))
            rdct= r.json()
            token = rdct['data'][0]['token']
            status = rdct['data'][0]['status']
            
            if status == 'complete' and token not in token_completed: 
                modified_url = load_response(token)
                print(token,status,modified_url)
                token_completed.append(token)
                urllib.request.urlretrieve(modified_url,'new_url/processed_image_{}'.format(j))
                j+=1
                
            if len(token_completed) == len(token_status.keys()):
                stptme = time.time()
                print ("All images processed at  ", stptme)
                return 1
                
            
        count+=1
        time.sleep(3)
    stptme = time.time()
    print ("Status poll has run out of interations ", stptme)      

In [99]:
def load_response(token):
    r = requests.get('https://photoai.lotlinx.com/images/{}'.format(token), auth=('testaccount5','984dc25e4e50'))
    r = r.json()
    modified_url = r['data'][0]['vehicles'][0]['images'][0]['modifiedUrl']
    #print(modified_url)
    return modified_url

In [100]:
poll_api()

Status poll has started at  1526760340.8724241
OBBh7FDrHJbfYgYudHAoPxTgR4wD5nCoPZFS06eAtiDF4B4PiD11can0th4SDNVu complete http://lotlinx-ml-images.s3.amazonaws.com/front/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863_7416_2_339187295.png
zzOhhc3JQOqumCTujipynSq1VX54muovfMNHcEFhhvJpvHa9TM9qPLtQm048nhwY complete http://lotlinx-ml-images.s3.amazonaws.com/front_3_quarter/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863_7416_339187295.png
NAaWW8jSkFBoEHqi39W5xBnuMQJnUUwm0QvG34Imt1cFmaS2wbrdGYEMhwAgktpu complete http://lotlinx-ml-images.s3.amazonaws.com/side/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863_7416_4_339187295.png
QawnDoVtCP5kpZX92NfX9iKof19tqJxzcu8UZMKAca4dI795q1M8hwVKkSuzSBqY complete http://lotlinx-ml-images.s3.amazonaws.com/side/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863_7416_3_339187295.png
nvBC8L9ad3PGti6Os5jrPLfdYjExmWytgF4kTgMROGF4UFPGMG3IUPwOlB8LFZ3Z complete http://lotlinx-ml-images.s3.amazonaws.com/side/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863

1